In [35]:
# FEMA Disaster Cost Forecasting

In [2]:
import pandas as pd
import requests
import os

In [3]:
'''
High Level Steps
---------------
Data Collection
Data Organization
Data Definition
Data Cleaning


project template
https://github.com/drivendata/cookiecutter-data-science
'''

'\nHigh Level Steps\n---------------\nData Collection\nData Organization\nData Definition\nData Cleaning\n\n\nproject template\nhttps://github.com/drivendata/cookiecutter-data-science\n'

In [4]:
#Data Collection

#Top level FEMA data sets
# https://www.fema.gov/about/openfema/data-sets

#OpenFEMA Dataset: FEMA Web Disaster Declarations - v1
# info - https://www.fema.gov/openfema-data-page/fema-web-disaster-declarations-v1
# data - https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations.csv
disasterInfoUrl = 'https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations.csv'

#OpenFEMA Dataset: FEMA Web Disaster Summaries - v1
# info - https://www.fema.gov/openfema-data-page/fema-web-disaster-summaries-v1
# data - https://www.fema.gov/api/open/v1/FemaWebDisasterSummaries.csv
disasterCostUrl = 'https://www.fema.gov/api/open/v1/FemaWebDisasterSummaries.csv'

#ClimRR
# info https://climrr.anl.gov/

In [5]:
#Data Download location
rawDataDir = '../data/raw/'
femaInfoPath = rawDataDir + 'FemaWebDisasterDeclarations.csv'
femaCostPath = rawDataDir + 'FemaWebDisasterSummaries.csv'

#Download Disaster Info data
r = requests.get(disasterInfoUrl)
with open(femaInfoPath, 'wb') as f:
    f.write(r.content)

#Download Disaster Cost data
r = requests.get(disasterCostUrl)
with open(femaCostPath, 'wb') as f:
    f.write(r.content)

In [6]:
#Data Loading

#Load FEMA disaster info data
femaInfo = pd.read_csv(femaInfoPath)

#Load FEMA disaster cost (federal financial assistance) data
femaCosts = pd.read_csv(femaCostPath)

In [40]:
#check loaded data shape
print("femaInfo rows, cols: " + str(femaInfo.shape))
print("femaCosts rows, cols: " + str(femaCosts.shape))
#femaCosts.columns

femaInfo rows, cols: (4865, 20)
femaCosts rows, cols: (3621, 14)


In [43]:
#check the head - fema info
femaInfo.head()

,disasterNumber,declarationDate,disasterName,incidentBeginDate,incidentEndDate,declarationType,stateCode,stateName,incidentType,entryDate,updateDate,closeoutDate,region,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,id,hash,lastRefresh
0,4734,2023-08-31T00:00:00.000Z,HURRICANE IDALIA,2023-08-27T00:00:00.000Z,2023-09-04T00:00:00.000Z,Major Disaster,FL,Florida,Hurricane,2023-08-31T00:00:00.000Z,2023-10-02T00:00:00.000Z,NaN,4,1.0,0.0,1.0,1.0,7039b9e8-8e40-411a-b3bd-ecb27b37d535,b2327cb14c124443d7e00b898be990718576195f,2023-10-02T22:21:25.390Z
1,4738,2023-09-07T00:00:00.000Z,HURRICANE IDALIA,2023-08-30T00:00:00.000Z,2023-08-30T00:00:00.000Z,Major Disaster,GA,Georgia,Hurricane,2023-09-07T00:00:00.000Z,2023-10-02T00:00:00.000Z,NaN,4,1.0,0.0,1.0,1.0,6f1316f1-788f-4763-9bc5-3c5d47e65f55,3b10fdf0c825cd0bc81e6f83955c85d9b32057e5,2023-10-02T22:41:25.957Z
2,4744,2023-10-06T00:00:00.000Z,SEVERE STORMS AND FLOODING,2023-08-03T00:00:00.000Z,2023-08-05T00:00:00.000Z,Major Disaster,VT,Vermont,Flood,2023-10-06T00:00:00.000Z,2023-10-06T00:00:00.000Z,NaN,1,0.0,0.0,1.0,1.0,f8a460e1-772c-4efe-91c3-bd7eed5df61e,314ca84418ff5016ef5c0fc7a1e32a6106b7e18b,2023-10-06T16:41:28.824Z
3,4745,2023-10-11T00:00:00.000Z,FLOODING,2023-06-01T00:00:00.000Z,2023-06-08T00:00:00.000Z,Major Disaster,MT,Montana,Flood,2023-10-11T00:00:00.000Z,2023-10-11T00:00:00.000Z,NaN,8,0.0,0.0,1.0,1.0,889564e8-549a-4123-88ba-a1d9b7b0261b,96fe8a62931b20e70e8a1779545c9467c66d29f8,2023-10-11T22:02:12.546Z
4,3404,2018-09-12T00:00:00.000Z,TROPICAL STORM OLIVIA,2018-09-09T00:00:00.000Z,2018-09-13T00:00:00.000Z,Emergency,HI,Hawaii,Hurricane,2018-09-12T00:00:00.000Z,2023-10-12T00:00:00.000Z,2023-10-11T00:00:00.000Z,9,0.0,0.0,1.0,0.0,5aa4296a-7d77-430f-a513-b7a433f6d305,87732b7d7d5a424fad84c04f5edf0bff4e860f50,2023-10-12T11:21:25.048Z


In [46]:
#check the head - fema costs
femaCosts.head()

,disasterNumber,totalNumberIaApproved,totalAmountIhpApproved,totalAmountHaApproved,totalAmountOnaApproved,totalObligatedAmountPa,totalObligatedAmountCatAb,totalObligatedAmountCatC2g,paLoadDate,iaLoadDate,totalObligatedAmountHmgp,hash,lastRefresh,id
0,3601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3de68baba960e69da445cf822d3dd859081fb34a,2023-10-09T23:02:26.341Z,faafecca-0f76-4fb8-8ffd-b6f46f3b712c
1,3602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58566c446fce5cabbd0c3412a6bb3daa4ada1993,2023-10-09T23:02:26.341Z,b74f0dc2-fab5-42b9-acf7-c94df14d85ad
2,1267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2167204.0,1ba476aa5b95b344e79f9ed5c0b1442849ccb5e0,2023-03-18T13:22:12.883Z,0f5ed8dd-d8e5-4328-9155-372206b47182
3,1270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319783.0,bb3c5cb8faf9c3e5606bd3f86ff09da517907068,2023-03-18T13:22:12.883Z,df214994-59d2-4075-b52c-894f7f4b358e
4,1290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200782.0,fbe5b8caa213cd388e1d7773b121c3c74559a5fb,2023-03-18T13:22:12.883Z,ffca81a5-0830-4f1c-b088-0ff4d5bb8e8a


In [61]:

#check that join fields are unique before join - femainfo.disasterNumber
print("disaster number is all unique(femaInfo)? " + str(len(femaInfo) == femaInfo['disasterNumber'].nunique()))

#check that join fields are unique before join - femacosts.disasterNumber
print("disaster number is all unique(femaCosts)? " + str(len(femaCosts) == femaCosts['disasterNumber'].nunique()))


disaster number is all unique(femaInfo)? True
disaster number is all unique(femaCosts)? True


In [63]:
#Display disaster categories
femaInfo.value_counts('incidentType')

incidentType
Fire                 1586
Severe Storm         1067
Flood                 885
Hurricane             442
Tornado               178
Snowstorm             171
Biological            167
Severe Ice Storm       74
Typhoon                63
Drought                46
Earthquake             36
Other                  34
Coastal Storm          31
Freezing               18
Mud/Landslide          12
Winter Storm           10
Tropical Storm          9
Toxic Substances        8
Fishing Losses          6
Dam/Levee Break         6
Volcanic Eruption       6
Human Cause             4
Tsunami                 3
Terrorist               2
Chemical                1
Name: count, dtype: int64

In [73]:
#Data Joining
#femaInfo & femaCosts 
print("Rows in femaInfo: " + str(len(femaInfo)))
print("Rows in femaInfo: " + str(len(femaCosts)))
print("*Note: There are more rows in the disaster [info] dataset than the disaster [costs]," + '\n' + "this suggests some disasters do not have cost info. Perhaps there was no claim to FEMA made.")

rows in femaInfo: 4865
rows in femaInfo: 3621
*Note: There are more rows in the disaster [info] dataset than the disaster [costs],
this suggests some disasters do not have cost info. Perhaps there was no claim to FEMA made.


In [91]:
#Join disaster info with disaster costs via the 'disaster info' column
femaMasterData = pd.merge(femaInfo, femaCosts, how='left', on='disasterNumber')

#check joined data set
print("(Rows, columns) in femaInfo" + '\n' + str(femaMasterData.shape))
femaMasterData.head()

(Rows, columns) in femaInfo
(4865, 33)


,disasterNumber,declarationDate,disasterName,incidentBeginDate,incidentEndDate,declarationType,stateCode,stateName,incidentType,entryDate,...,totalAmountOnaApproved,totalObligatedAmountPa,totalObligatedAmountCatAb,totalObligatedAmountCatC2g,paLoadDate,iaLoadDate,totalObligatedAmountHmgp,hash_y,lastRefresh_y,id_y
0,4734,2023-08-31T00:00:00.000Z,HURRICANE IDALIA,2023-08-27T00:00:00.000Z,2023-09-04T00:00:00.000Z,Major Disaster,FL,Florida,Hurricane,2023-08-31T00:00:00.000Z,...,26511441.07,NaN,NaN,NaN,NaN,2024-04-19T00:00:00.000Z,3210378.19,f7a0d72da7878b3cb0a79eaeba2a8befcd983b81,2024-04-20T00:44:25.472Z,1c190728-3c46-4d97-9aa8-5b38ef98094f
1,4738,2023-09-07T00:00:00.000Z,HURRICANE IDALIA,2023-08-30T00:00:00.000Z,2023-08-30T00:00:00.000Z,Major Disaster,GA,Georgia,Hurricane,2023-09-07T00:00:00.000Z,...,586041.75,NaN,NaN,NaN,NaN,2024-04-19T00:00:00.000Z,0.00,cabe17a05481dd3dcca1e3cc9f484a2a1ede7cc9,2024-04-20T00:44:25.472Z,c0a82a50-236f-40f5-a99b-b19ad2537520
2,4744,2023-10-06T00:00:00.000Z,SEVERE STORMS AND FLOODING,2023-08-03T00:00:00.000Z,2023-08-05T00:00:00.000Z,Major Disaster,VT,Vermont,Flood,2023-10-06T00:00:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,f2cb61543220cab0d3f7c550b688731b8aa8549a,2024-04-20T00:44:25.472Z,c413bde3-aa24-4d13-ae6b-5cb9fa62ffe9
3,4745,2023-10-11T00:00:00.000Z,FLOODING,2023-06-01T00:00:00.000Z,2023-06-08T00:00:00.000Z,Major Disaster,MT,Montana,Flood,2023-10-11T00:00:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,58b35da74cee2611259abc8fd95550b4276eccd4,2024-04-20T00:44:25.472Z,7b55e677-7c65-4131-9306-459c5e1e1a92
4,3404,2018-09-12T00:00:00.000Z,TROPICAL STORM OLIVIA,2018-09-09T00:00:00.000Z,2018-09-13T00:00:00.000Z,Emergency,HI,Hawaii,Hurricane,2018-09-12T00:00:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
#Data organization
# Project based on the cookiecutter data science project template. #cookiecutterdatascience
# URL - https://drivendata.github.io/cookiecutter-data-science/

In [98]:
#Data definition
"""
Column names
■ Data types
■ Description of the columns
■ Counts and percents unique values
■ Ranges of values
■ Calc Summary statistics
"""

#Data types
femaMasterData.info()

#TODO: lots of object columns that should be either datetime or str

#df type conversion example
#df["B"] = df["A"].astype("category")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4865 entries, 0 to 4864
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   disasterNumber              4865 non-null   int64  
 1   declarationDate             4865 non-null   object 
 2   disasterName                4865 non-null   object 
 3   incidentBeginDate           4865 non-null   object 
 4   incidentEndDate             4599 non-null   object 
 5   declarationType             4865 non-null   object 
 6   stateCode                   4865 non-null   object 
 7   stateName                   4865 non-null   object 
 8   incidentType                4865 non-null   object 
 9   entryDate                   4865 non-null   object 
 10  updateDate                  4865 non-null   object 
 11  closeoutDate                3910 non-null   object 
 12  region                      4865 non-null   int64  
 13  ihProgramDeclared           4614 